In [96]:
import cv2
import numpy as np

In [97]:
def align_images(image1: np.ndarray, image2: np.ndarray) -> np.ndarray:
    """
    Aligns image2 to image1 using SIFT and homography.

    Args:
        image1 (np.ndarray): The reference image.
        image2 (np.ndarray): The image to be aligned.

    Returns:
        np.ndarray: The aligned version of image2.
    """
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

    # Match descriptors using FLANN-based matcher
    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=50))
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Filter matches using Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    # Extract matched keypoints
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches])
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches])

    # Compute homography
    homography, _ = cv2.findHomography(points2, points1, cv2.RANSAC)

    # Warp image2 to align with image1
    height, width, _ = image1.shape
    aligned_image2 = cv2.warpPerspective(image2, homography, (width, height))

    return aligned_image2

In [98]:
image1 = cv2.imread('pictures/20250611_232336.jpg')
image2 = cv2.imread('pictures/20250613_110137.jpg')

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.show()

In [100]:
def histogram_equalization_gray(image):
    if len(image.shape) == 3:  # Check if the image has multiple channels
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image
    return cv2.equalizeHist(gray_image)

def histogram_equalization_rgb(image):
    b_channel, g_channel, r_channel = cv2.split(image)
    b_equalized = cv2.equalizeHist(b_channel)
    g_equalized = cv2.equalizeHist(g_channel)
    r_equalized = cv2.equalizeHist(r_channel)
    return cv2.merge((b_equalized, g_equalized, r_equalized))

def histogram_equalization_color(image, tile_size=(8,8)):
    # Convert the image to YCrCb color space
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCR_CB)
    y_channel, cr_channel, cb_channel = cv2.split(ycrcb_image)
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the Y channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=tile_size)
    y_equalized = clahe.apply(y_channel)
    ycrcb_equalized = cv2.merge((y_equalized, cr_channel, cb_channel))
    return cv2.cvtColor(ycrcb_equalized, cv2.COLOR_YCR_CB2BGR)

In [ ]:
plt.imshow(cv2.cvtColor(histogram_equalization_color(image1), cv2.COLOR_BGR2RGB))
plt.show()